In [1]:
offset = 0
limit = 3000
period = '7y' # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max

In [2]:
import pandas as pd

data = pd.read_csv("Datasets/nasdaq_screener_1698005734880.csv", sep=',')
#data_clean = data[data['Test Issue'] == 'N']
symbols = data['Symbol'].tolist()
print('total number of symbols traded = {}'.format(len(symbols)))

total number of symbols traded = 72


In [4]:
import yfinance as yf
import os, contextlib
import pymongo as pm

In [5]:
client = pm.MongoClient("localhost", 27017)
db = client["tarea"]

In [6]:
%%time

limit = limit if limit else len(symbols)
end = min(offset + limit, len(symbols))
is_valid = [False] * len(symbols)
# force silencing of verbose API
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        for i in range(offset, end):
            s = symbols[i]
            data = yf.download(s, period=period)
            if len(data.index) == 0:
                continue
        
            is_valid[i] = True
            data.to_json('Datasets/hist/{}.json'.format(s), orient='columns', date_format='iso' )
            data.to_csv('Datasets/hist/{}.csv'.format(s))
            collectionName = "stock-" + format(s)
            collection = db.create_collection( collectionName )
            #collection.insert_one( data.to_dict('dict') )
            
print('Total number of valid symbols downloaded = {}'.format(sum(is_valid)))

Total number of valid symbols downloaded = 72
CPU times: total: 28.6 s
Wall time: 59 s


In [45]:
data
data.to_dict('dict')

{'Open': {Timestamp('2016-10-24 00:00:00'): 29.274999618530273,
  Timestamp('2016-10-25 00:00:00'): 29.487499237060547,
  Timestamp('2016-10-26 00:00:00'): 28.577499389648438,
  Timestamp('2016-10-27 00:00:00'): 28.84749984741211,
  Timestamp('2016-10-28 00:00:00'): 28.467500686645508,
  Timestamp('2016-10-31 00:00:00'): 28.412500381469727,
  Timestamp('2016-11-01 00:00:00'): 28.364999771118164,
  Timestamp('2016-11-02 00:00:00'): 27.850000381469727,
  Timestamp('2016-11-03 00:00:00'): 27.7450008392334,
  Timestamp('2016-11-04 00:00:00'): 27.13249969482422,
  Timestamp('2016-11-07 00:00:00'): 27.520000457763672,
  Timestamp('2016-11-08 00:00:00'): 27.577499389648438,
  Timestamp('2016-11-09 00:00:00'): 27.469999313354492,
  Timestamp('2016-11-10 00:00:00'): 27.772499084472656,
  Timestamp('2016-11-11 00:00:00'): 26.780000686645508,
  Timestamp('2016-11-14 00:00:00'): 26.927499771118164,
  Timestamp('2016-11-15 00:00:00'): 26.642499923706055,
  Timestamp('2016-11-16 00:00:00'): 26.67499